# Compare SECP256k1, SHA-256, RIPEMD160 and BASE58

In [17]:
!python3 -m pip install siphash24 pyopencl base58 ecdsa pycryptodome Cython cryptography

In [6]:
import hashlib
import time
from ecdsa import SECP256k1, SigningKey
import binascii
import base58
import time
import os
import numpy as np
import pyopencl as cl
from Crypto.Hash import SHA256
import cryptography
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.hashes import Hash
from multiprocessing import Pool

values = [os.urandom(32) for _ in range(0, 10_000)]

## SECP256k1

In [2]:
start_time = time.time()

for value in values:
    result = SigningKey.from_string(value, curve=SECP256k1)
    result = result.get_verifying_key()
    result.to_string("compressed")

end_time = time.time()

print(f"Time taken (SECP256k1): {end_time - start_time:.2f} seconds")


Time taken (SECP256k1): 6.18 seconds


## SHA-256
### using hashlib

In [8]:
start_time = time.time()

for value in values:
    result = hashlib.sha256(value).digest()
    result = hashlib.sha256(result).digest()
    result = hashlib.sha256(result).digest()

end_time = time.time()

print(f"Time taken (3 x SHA-256): {end_time - start_time:.2f} seconds")

Time taken (3 x SHA-256): 0.03 seconds


### using Crypto.Hash

In [12]:
# Concatenate all strings to avoid per-call overhead
data = b''.join([f"string_{i}".encode('utf-8') for i in range(100_000)])

start_time = time.time()

h = SHA256.new()
h.update(data)  # Process all input data in one call
digest = h.digest()

end_time = time.time()
print(f"Time taken (batch processing): {end_time - start_time:.2f} seconds")

Time taken (batch processing): 0.00 seconds


### using PyopenCL

In [16]:
# OpenCL kernel for SHA-256 (simplified for now)
sha256_kernel = """
__kernel void sha256(__global const char* input, __global char* output) {
    int gid = get_global_id(0);
    // Implement SHA-256 logic here (currently just copying input to output)
    for (int i = 0; i < 64; ++i) {  // Assuming 64 bytes per input
        output[gid * 64 + i] = input[gid * 64 + i];
    }
}
"""

# Number of input strings
num_strings = 10_000_000
max_string_length = 64  # Fixed size for each input string

# Initialize OpenCL context and queue
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.Context([device])
queue = cl.CommandQueue(context)

# Generate the input data: fixed-length strings
strings = [f"string_{i}"[:max_string_length].ljust(max_string_length, '\0') for i in range(num_strings)]
input_data = np.array([list(s.encode('utf-8')) for s in strings], dtype=np.uint8).flatten()

# Initialize output array
output_data = np.zeros_like(input_data, dtype=np.uint8)

# Create OpenCL buffers
input_buffer = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=input_data)
output_buffer = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, output_data.nbytes)

# Compile OpenCL program
program = cl.Program(context, sha256_kernel).build()

# Run the OpenCL kernel
start_time = time.time()
program.sha256(queue, (num_strings,), None, input_buffer, output_buffer)
cl.enqueue_copy(queue, output_data, output_buffer)
queue.finish()
end_time = time.time()

print(f"Time taken (OpenCL GPU): {end_time - start_time:.2f} seconds")

# Optional: print first 10 outputs for verification (truncated for brevity)
for i in range(10):
    print(f"Output {i}: {output_data[i * max_string_length:(i + 1) * max_string_length].tobytes().decode('utf-8', errors='ignore')}")


Time taken (OpenCL GPU): 0.25 seconds
Output 0: string_0                                                        
Output 1: string_1                                                        
Output 2: string_2                                                        
Output 3: string_3                                                        
Output 4: string_4                                                        
Output 5: string_5                                                        
Output 6: string_6                                                        
Output 7: string_7                                                        
Output 8: string_8                                                        
Output 9: string_9                                                        


### using cryptography

In [8]:
data = b''.join([f"string_{i}".encode('utf-8') for i in range(10000000)])

start_time = time.time()

# Use cryptography's SHA-256 implementation
digest = Hash(hashes.SHA256(), backend=default_backend())
digest.update(data)
final_hash = digest.finalize()

end_time = time.time()

print(f"Time taken with cryptography (OpenSSL backend): {end_time - start_time:.2f} seconds")

Time taken with cryptography (OpenSSL backend): 0.19 seconds


## RIPEMD160

In [8]:
start_time = time.time()

for value in values:
    result = hashlib.new('ripemd160', value).digest()

end_time = time.time()

print(f"Time taken (RIPEMD160): {end_time - start_time:.2f} seconds")

Time taken (RIPEMD160): 0.01 seconds


## BASE58

In [9]:
start_time = time.time()

for value in values:
    result = base58.b58encode(value).decode()

end_time = time.time()

print(f"Time taken (BASE58): {end_time - start_time:.2f} seconds")

Time taken (BASE58): 0.09 seconds


## All together

In [63]:
def private_to_public(private_key_bytes):
    private_key = SigningKey.from_string(private_key_bytes, curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    return public_key.to_string("compressed")

def public_key_to_address(compressed_public_key_bytes):
    sha256_hash = hashlib.sha256(compressed_public_key_bytes).digest()
    ripemd160_hash = hashlib.new('ripemd160', sha256_hash).digest()
    versioned_payload = b'\x00' + ripemd160_hash
    checksum = hashlib.sha256(hashlib.sha256(versioned_payload).digest()).digest()[:4]
    payload_with_checksum = versioned_payload + checksum
    return base58.b58encode(payload_with_checksum).decode()

def generate_key(private_key):
    private_key_hex = '{:064x}'.format(private_key)
    private_key_bytes = binascii.unhexlify(private_key_hex)

    compressed_public_key = private_to_public(private_key_bytes)
    bitcoin_address = public_key_to_address(compressed_public_key)
    
    return (private_key_hex, binascii.hexlify(compressed_public_key).decode(), bitcoin_address)

start_time = time.time()

for value in values:
    generate_key(int.from_bytes(value, byteorder='big'))

end_time = time.time()

print(f"Time taken (all together): {end_time - start_time:.2f} seconds")

Time taken (all together): 4.82 seconds
